Темы: 
- метрики при ранжировании
<!-- - MLSysDes в задачах/хакатонах (задача на ТЗ)  -->

# 1 Задачи ранжирования
Разбивается на 2 подзадачи:
- выбрать/отобрать нужных кандидатов на ранжирование,
- осуществить процесс ранжирования


### 1.1 Источники

[Материалы][1] и [лекция][2] Воронцова

[Yandex ml book статья][3]

[Итмо][4]

<!-- [Ким Фальк???] -->

[1]: http://www.machinelearning.ru/wiki/index.php?title=%D0%9C%D0%B0%D1%88%D0%B8%D0%BD%D0%BD%D0%BE%D0%B5_%D0%BE%D0%B1%D1%83%D1%87%D0%B5%D0%BD%D0%B8%D0%B5_(%D0%BA%D1%83%D1%80%D1%81_%D0%BB%D0%B5%D0%BA%D1%86%D0%B8%D0%B9%2C_%D0%9A.%D0%92.%D0%92%D0%BE%D1%80%D0%BE%D0%BD%D1%86%D0%BE%D0%B2)#.D0.9E.D0.B1.D1.83.D1.87.D0.B5.D0.BD.D0.B8.D0.B5_.D1.80.D0.B0.D0.BD.D0.B6.D0.B8.D1.80.D0.BE.D0.B2.D0.B0.D0.BD.D0.B8.D1.8E

[2]: https://www.youtube.com/watch?v=kQ5PeshAO1w

[3]: https://education.yandex.ru/handbook/ml/article/zadacha-ranzhirovaniya

[4]: https://neerc.ifmo.ru/wiki/index.php?title=%D0%A0%D0%B0%D0%BD%D0%B6%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5

### 1.2 Постановка задачи

X - множество объектов.

$X^{l}  = \{ x_1, ..., x_{l} \}$ - обучающая выборка.

Есть набор пар $(x_{i}, x_{j})$ для которых известно, что  
$x_{i}$ в каком-то смысле лучше/релевантнее $x_{j}$, но это известно не для всех двух элементов из X. (Частичный порядок) 

**Задача**

Построить ранжирующую функцию $a : X \rightarrow \mathbb{R}$ такую, что для тех пар на которых определен порядок $i \prec j  \Rightarrow a(x_i)<a(x_j)$ 




### 1.3 Примеры

#### Задача ранжирования поисковой выдачи
$D$ — коллекция текстовых документов.

$Q$ — множество запросов.

$D_q \subseteq  D$  — множество документов, найденных по запросу q.

$X = Q \times D$  — объектами являются пары (запрос, документ): $x≡(q,d)$,$q∈Q$,$d∈D_q$.

$Y$ — упорядоченное множество рейтингов.

$y:X→Y$  — оценки релевантности, поставленные асессорами (экспертами): чем выше оценка $y(q,d)$ , тем релевантнее документ $d$ по запросу $q$.

Правильный порядок определен только между документами, найденными по одному и тому же запросу $q$: $(q,d)≺(q,\hat{d})⇔y(q,d)<y(q,\hat{d})$
.

Релевантные ответы запросу $q$ — это список документов $d$, упорядоченных с помощью функции ранжирования $a(q,d)$.

# 2 Виды проведения обучения 
- поточечный (pointwise)
- попарный (pairwise)
- списочный (listwise)

Выбор метода зависит от качества ранжирования данных. Теоретически, списочный подход считается наилучшим, однако, на практике, лучше всего работает попарный подход. (источник?)

### 2.1 Поточечный подход

Оценка релевантности $a(q,d)$ считается для каждого объекта независимо от других пар $(q,d)$

Если речь идёт о задаче ранжирования поисковой выдачи, то пусть асессор поставил какую-то оценку $y$ каждой паре (запрос, документ).
$$ (q, d) \rightarrow y$$

Эта оценка и будет предсказываться. 

При этом никак не учитывается, что **нужно предсказать порядок объектов**, а не оценки. Этот подход является простым в том смысле, что в нём используются уже известные методы. Например, можно предсказывать оценки с использованием линейной регрессии и квадратичной ошибки:

$$ \sum_{i=1}^{l}(a(q_i,d_i)−y(q_i,d_i))^2 \rightarrow min$$




Известно, как решать такую задачу, и таким образом будет получена релевантность. Далее по выходам модели можно ранжировать объекты.

### 2.2 Попарный подход

В попарном подходе используются знания об устройстве целевой переменной. Модель строится сведением к минимуму количества дефектных пар, то есть таких, в которых моделью был предсказан неправильный порядок:

$$ \sum_{x_i < x_j}[a(x_j)- a(x_i)<0] \rightarrow min$$



К сожалению, этот функционал дискретный (в него входят индикаторы), поэтому невозможно его минимизировать. Однако можно действовать так же, как и с классификаторами: оценить функционал сверху.

Можно считать, что разница между объектами $a(x_j)−a(x_i)$  — это отступ $M$, и задать некоторую гладкую функцию $L(M)$:

$$ \sum_{x_i < x_j}L(a(x_j)- a(x_i)) \rightarrow min$$



Если использовать функцию как в логистической регрессии 
$$L(M)=log(1+e^{−M}),$$
то полученный метод называется *RankNet*. Затем можно решать задачу, например, с помощью стохастического градиентного спуска.

# 3 Метрики качества ранжирования
- precision@k, 
- recall@k,
- AP@k,
- MAP@k,
- nDCG

### 3.1 Точность ранжирования


## Примеры вычисления метрик 

# Дополнительно


### Как осуществляется отбор кандидатов +/- 
- случайно,
- потом среди Ложноположительных которые (y = -1, a(x) = 1)


### Другие парные функции потерь

- [PairwiseLogisticLoss][1]
- [PairwiseHingeLoss][2]
- [PairwiseMSELoss][3]

[1]: https://www.tensorflow.org/ranking/api_docs/python/tfr/keras/losses/PairwiseLogisticLoss
[2]: https://www.tensorflow.org/ranking/api_docs/python/tfr/keras/losses/PairwiseHingeLoss
[3]: https://www.tensorflow.org/ranking/api_docs/python/tfr/keras/losses/PairwiseMSELoss

### Какие есть особенности деплоя в задачах ранжирования?